In [26]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import pi
from PIL import Image
from pathlib import Path

plt.rcParams['axes.axisbelow'] = True

In [31]:
class WaveEq:
    def __init__(self, r_min = 0.0, r_max = 1.8, c = 1.5, a = 0.6, b = 1.2, I = 100, CFL = 0.5, d = 3, img_path = 'default', gif_path = 'default'):
        self.d = d
        self.I = I
        self.r_min = r_min
        self.r_max = r_max
        self.c = c
        self.a = a
        self.b = b
        self.i = np.arange(self.I + 2)
        self.h = (self.r_max - self.r_min) / I
        self.r_i = np.array([self.r_min + (iter - 0.5) * self.h for iter in self.i])
        self.T = 3
        self.CFL = CFL
        self.tau = self.CFL * self.h / self.c
        self.t_i = np.linspace(0, self.T, int(self.T / self.tau + 1))
        self.u_p = self.v0(self.r_i)
        self.dv0_i = self.dv0(self.r_i)
        self.du_t = lambda x: 0.0
        self.du_t_i = self.du_t(self.r_i)
        self.f_i = np.array([0.0 for i in self.i])

        if self.d == 1:
            self.d2u_t_i = np.power(self.c, 2) * self.d2u_factor_1d(self.r_i) - self.f_i
            self._ylim = [-0.5, 1.5]
        elif self.d == 2:
            self.d2u_t_i = np.power(self.c, 2) * (1 / np.power(self.r_i, self.d - 1)) * self.d2u_factor_2d(self.r_i) - self.f_i
            self._ylim = [-2, 3]
        elif self.d == 3:
            self.d2u_t_i = np.power(self.c, 2) * (1 / np.power(self.r_i, self.d - 1)) * self.d2u_factor_3d(self.r_i) - self.f_i
            self._ylim = [-4, 4]


        self.u = self.u_p + self.tau * self.du_t_i + np.power(self.tau, 2) / 2 * self.d2u_t_i
        self.u_n = self.u.copy()

        self.curr_time_step = 0
        self.curr_time = 0

        self._images_path = Path('./img/' + img_path)
        self._gifs_path = Path('./gifs/' + gif_path)
        if not self._images_path.is_dir():
            self._images_path.mkdir(parents=True, exist_ok=True)
        if not self._gifs_path.is_dir():
            self._gifs_path.mkdir(parents=True, exist_ok=True)
        

    def printParams(self):
        print('d\t', self.d)
        print('I\t', self.I)
        print('r_min\t', self.r_min)
        print('r_max\t', self.r_max)
        print('c\t', self.c)
        print('a\t', self.a)
        print('b\t', self.b)
        print('i\t', self.i)
        print('h\t', self.h)
        print('r_i\t', self.r_i)
        print('T\t', self.T)
        print('CFL\t', self.CFL)
        print('tau\t', self.tau)
        print('t_i\t', self.t_i)
        print('U\t', self.u)
        print('U_prev\t', self.u_p)
        print('U_next\t', self.u_n)


    def solutionStep(self):
        if(self.curr_time_step == 0):
            plt.figure(figsize=(10, 6))
            plt.scatter(self.r_i, self.u_p, color = 'black', s = 10)
            plt.plot(self.r_i, self.u_p, color = 'black')
            plt.ylim(self._ylim)
            plt.title(f'Решение волнового уравнения\nt = {self.t_i[0]:.4f}, CFL = {self.CFL}, h = {self.h}, {self.d}D', fontsize=14)
            plt.grid(which='major', linestyle='-')
            plt.grid(which='minor', linestyle='--')
            plt.minorticks_on()
            
            plt.savefig(self._images_path.joinpath(f'img_{0}.png'), 
                    transparent = False,  
                    facecolor = 'white'
                    )
            plt.close()

            plt.figure(figsize=(10, 6))
            plt.scatter(self.r_i, self.u, color = 'black', s = 10)
            plt.plot(self.r_i, self.u, color = 'black')
            plt.ylim(self._ylim)
            plt.title(f'Решение волнового уравнения\nt = {self.t_i[1]:.4f}, CFL = {self.CFL}, h = {self.h}, {self.d}D', fontsize=14)
            plt.grid(which='major', linestyle='-')
            plt.grid(which='minor', linestyle='--')
            plt.minorticks_on()
            plt.savefig(self._images_path.joinpath(f'img_{1}.png'), 
                    transparent = False,  
                    facecolor = 'white'
                    )
            plt.close()

            self.curr_time_step = 2
            self.curr_time = 2 * self.tau

        elif self.curr_time <= self.T:
            for i in range(1, self.I + 1):
                self.u_n[i] = 2 * self.u[i] - self.u_p[i] + (np.power(self.tau * self.c, 2) / (np.power(self.r_i[i], self.d - 1) * self.h)) \
                                * (np.power((self.r_i[i] + self.h / 2), self.d - 1) * ((self.u[i + 1] - self.u[i]) / self.h) - np.power((self.r_i[i] - self.h / 2), self.d - 1) * ((self.u[i] - self.u[i - 1]) / self.h)) \
                                + np.power(self.tau, 2) * self.f_i[i]

            self.u_n[0] = self.u_n[1]
            self.u_n[self.I + 1] = self.u_n[self.I]
        
            plt.figure(figsize=(10, 6))
            plt.scatter(self.r_i, self.u_n, color = 'black', s = 10)
            plt.plot(self.r_i, self.u_n, color = 'black')
            plt.ylim(self._ylim)
            plt.title(f'Решение волнового уравнения\nt = {self.t_i[self.curr_time_step]:.4f}, CFL = {self.CFL}, h = {self.h}, {self.d}D', fontsize=14)
            plt.grid(which='major', linestyle='-')
            plt.grid(which='minor', linestyle='--')
            plt.minorticks_on()
            plt.savefig(self._images_path.joinpath(f'img_{self.curr_time_step}.png'), 
                    transparent = False,  
                    facecolor = 'white'
                    )
            plt.close()

            self.u_p = self.u.copy()
            self.u = self.u_n.copy()
            self.curr_time_step += 1
            self.curr_time += self.tau

        else:
            raise Exception('Время на текущей итерации больше времени расчёта')
        
    def createGif(self):
        p = self._images_path.as_posix()
        file_count = sum(1 for file in self._images_path.iterdir() if file.is_file())
        image_path_list = [p + f'/img_{i}.png' for i in range(file_count)]
        image_list = [Image.open(file) for file in image_path_list]
        image_list[0].save(
                self._gifs_path.joinpath(f'waveEquation{self.d}D-h{self.h}-CFL{self.CFL}.gif'),
                save_all=True,
                append_images=image_list[1:],
                duration=self.T / len(self.t_i),
                loop=0)

    def v0(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b], 
                            [lambda r: 0,  
                            lambda r: np.exp((-4 * np.power((2 * r - (self.a + self.b)), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - (self.a + self.b)), 2))),
                            lambda r: 0])
    
    def dv0(self, r):
        return np.piecewise(r, 
                            [r <= self.a, (r > self.a) & (r < self.b), r >= self.b],
                            [lambda r: 0,
                            lambda r: (np.power((self.a - self.b), 2) * (self.a + self.b - 2*r)) \
                                * np.exp((-4 * np.power((2 * r - (self.a + self.b)), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - (self.a + self.b)), 2))) \
                                / (np.power((self.a - r), 2) * np.power((self.b - r), 2)),
                            lambda r: 0])
    
    def d2u_factor_1d(self, r):
        return np.piecewise(r, 
                        [r <= self.a, (r > self.a) & (r < self.b), r >= self.b],
                        [lambda r: 0,
                        lambda r: np.power(self.a - self.b, 2) / (np.power(self.a - r, 2) * np.power(self.b - r, 2)) \
                            * (self.a**4 + 2*self.a**3*self.b - 6*self.a**3*r - 6*self.a**2*self.b*r + 12*self.a**2*r**2 + 2*self.a*self.b**3 - 6*self.a*self.b**2*r + 12*self.a*self.b*r**2 - 12*self.a*r**3 \
                            + self.b**4 - 6*self.b**3*r + 12*self.b**2*r**2 - 12*self.b*r**3 + 6*r**4) \
                            * np.exp((-4 * np.power((2 * r - (self.a + self.b)), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - (self.a + self.b)), 2))),
                        lambda r: 0])
    
    def d2u_factor_2d(self, r):
        return np.piecewise(r, 
                        [r <= self.a, (r > self.a) & (r < self.b), r >= self.b],
                        [lambda r: 0,
                        lambda r: np.power(self.a - self.b, 2) / (np.power(self.a - r, 2) * np.power(self.b - r, 2)) \
                            * (self.a**4*r + self.a**3*self.b**2 - 5*self.a**3*r**2 + self.a**2*self.b**3 - 6*self.a**2*self.b**2*r + 3*self.a**2*self.b*r**2 + 8*self.a**2*r**3 + 3*self.a*self.b**2*r**2 - 7*self.a*r**4 \
                            + self.b**4*r - 5*self.b**3*r**2 + 8*self.b**2*r**3 - 7*self.b*r**4 + 4*r**5) \
                            * np.exp((-4 * np.power((2 * r - (self.a + self.b)), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - (self.a + self.b)), 2))),
                        lambda r: 0])
    
    def d2u_factor_3d(self, r):
        return np.piecewise(r, 
                        [r <= self.a, (r > self.a) & (r < self.b), r >= self.b],
                        [lambda r: 0,
                        lambda r: r * np.power(self.a - self.b, 2) / (np.power(self.a - r, 2) * np.power(self.b - r, 2)) \
                            * (self.a**4*r + 2*self.a**3*self.b**2 - 2*self.a**3*self.b*r - 4*self.a**3*r**2 + 2*self.a**2*self.b**3 - 12*self.a**2*self.b**2*r + 12*self.a**2*self.b*r**2 + 4*self.a**2*r**3 - 2*self.a*self.b**3*r \
                            + 12*self.a*self.b**2*r**2 - 12*self.a*self.b*r**3 - 2*self.a*r**4 + self.b**4*r - 4*self.b**3*r**2 + 4*self.b**2*r**3 - 2*self.b*r**4 + 2*r**5) \
                            * np.exp((-4 * np.power((2 * r - (self.a + self.b)), 2)) / (np.power((self.b - self.a), 2) - np.power((2 * r - (self.a + self.b)), 2))),
                        lambda r: 0])

In [ ]:
wave1d = WaveEq(I = 180, d = 1, img_path='1d', gif_path='1d')
while(wave1d.curr_time <= wave1d.T):
    wave1d.solutionStep()

In [ ]:
wave2d = WaveEq(I = 180, d = 2, img_path='2d', gif_path='2d')
while(wave2d.curr_time <= wave2d.T):
    wave2d.solutionStep()

In [ ]:
wave3d = WaveEq(I = 180, d = 3, img_path='3d', gif_path='3d')
while(wave3d.curr_time <= wave3d.T):
    wave3d.solutionStep()

In [32]:
wave1d.createGif()
wave2d.createGif()
wave3d.createGif()